In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score,recall_score,f1_score

In [2]:
df = pd.read_csv('data/secondary-prediction/secondary_prediction_players_statistics.csv')
df.groupby(['game']).size().sort_values(ascending=True)

game
ESPORTSTMNT01_2690210    2
ESPORTSTMNT03_2564747    2
ESPORTSTMNT03_2564748    2
ESPORTSTMNT03_2564761    2
ESPORTSTMNT03_2564782    2
                        ..
ESPORTSTMNT01_2782376    2
ESPORTSTMNT01_2782404    2
ESPORTSTMNT01_2782430    2
ESPORTSTMNT01_2782842    2
NA1_4217912820           2
Length: 2834, dtype: int64

In [3]:
y = df['firstTower'].copy()
X = df.drop(['game', 'firstBlood', 'kills', 'deaths', 'firstTower', 'firstHerald', 'dragons', 'barons', 'inhibitors', 'towers', 'heralds'], axis=1)
X

,flagSide,topGP,topWR,topKDA,jungleGP,jungleWR,jungleKDA,midGP,midWR,midKDA,carryGP,carryWR,carryKDA,suppGP,suppWR,suppKDA
0,1,3,0.67,4.0,0,0.00,0.0,12,0.42,4.2,1,0.00,4.0,1,1.00,9.0
1,0,6,0.50,5.1,4,0.50,4.7,0,0.00,0.0,1,0.00,1.3,11,0.36,2.4
2,1,4,0.25,2.2,2,0.00,1.2,3,0.33,4.1,0,0.00,0.0,0,0.00,0.0
3,0,4,0.00,1.5,3,0.33,1.1,2,0.50,4.2,4,0.50,3.0,0,0.00,0.0
4,1,3,0.00,1.6,0,0.00,0.0,1,0.00,0.7,5,0.40,3.1,0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5663,0,0,0.00,0.0,9,0.33,3.2,3,0.67,8.3,51,0.51,4.5,3,0.67,7.3
5664,1,0,0.00,0.0,7,0.71,3.1,0,0.00,0.0,12,0.50,4.1,27,0.59,3.1
5665,0,5,0.40,2.5,7,0.71,5.5,2,0.00,1.8,6,0.67,4.7,5,0.20,1.3
5666,1,16,0.75,4.8,9,0.78,4.7,4,0.50,1.9,24,0.63,5.4,2,1.00,4.6


In [4]:
def preprocess_input(X,y):
    X = X.copy()
    X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=42, stratify=y)
    scaler = StandardScaler()   
    print(X_train) 
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train,X_test,y_train,y_test
X_train,X_test,y_train,y_test = preprocess_input(X,y)

      flagSide  topGP  topWR  topKDA  jungleGP  jungleWR  jungleKDA  midGP  \
1634         1     12   0.50     3.8         6      1.00        4.1      0   
2557         0      0   0.00     0.0         1      0.00        2.5      0   
3543         0      4   1.00     7.3        28      0.64        5.2     33   
1214         1      1   1.00     6.5         4      1.00        5.4      0   
1219         0      1   0.00     1.0         1      0.00        1.8      0   
...        ...    ...    ...     ...       ...       ...        ...    ...   
2494         1      0   0.00     0.0         0      0.00        0.0     14   
3229         0      0   0.00     0.0        24      0.38        2.4      2   
2577         0     24   0.46     2.9         0      0.00        0.0      5   
2513         0      0   0.00     0.0         2      1.00        7.4      0   
3967         0      8   0.50     4.0         7      0.43        2.6      2   

      midWR  midKDA  carryGP  carryWR  carryKDA  suppGP  suppWR

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier

models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Support Vector Machine (Linear Kernel)': LinearSVC(),
    'Support Vector Machine (RBF Kernel)': SVC(),
    'Decission Tree': DecisionTreeClassifier(),
    'Adaboost': AdaBoostClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting Classifier': GradientBoostingClassifier()
}

GBCmodel = GradientBoostingClassifier()
GBCmodel.fit(X_train,y_train)


for name, model in models.items():
  model.fit(X_train,y_train)
  print(name + ' trained')

Logistic Regression trained


C:\Users\linco\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Support Vector Machine (Linear Kernel) trained
Support Vector Machine (RBF Kernel) trained
Decission Tree trained
Adaboost trained
Random Forest trained
Gradient Boosting Classifier trained


In [6]:
scores_list = []

for name,model in models.items():    
    scores_list.append({
    'Model': name,
    'Accuracy': accuracy_score(y_test,model.predict(X_test)),
    'Precision':  precision_score(y_test,model.predict(X_test)),
    'Recall': recall_score(y_test,model.predict(X_test)),
    'F1-Score': f1_score(y_test,model.predict(X_test))
    })
scores = pd.DataFrame(scores_list)

In [7]:
scores

,Model,Accuracy,Precision,Recall,F1-Score
0,Logistic Regression,0.542328,0.543321,0.530864,0.537021
1,Support Vector Machine (Linear Kernel),0.542328,0.543321,0.530864,0.537021
2,Support Vector Machine (RBF Kernel),0.559083,0.556876,0.578483,0.567474
3,Decission Tree,0.528219,0.527778,0.536155,0.531934
4,Adaboost,0.529101,0.530275,0.509700,0.519784
5,Random Forest,0.539683,0.538860,0.550265,0.544503
6,Gradient Boosting Classifier,0.537037,0.535836,0.553792,0.544666
